In [3]:
from cag.graph_framework.components import AnalyzerBase
from cag.graph_framework.graph.base_graph import BaseGraph
from pyArango.graph import Graph, EdgeDefinition
from pyArango.query import AQLQuery
from cag.utils.config import Config


class KnowledgeGraph(BaseGraph):
    _edgeDefinitions = [EdgeDefinition('GenericEdge', fromCollections=[
                                       'GenericNode'], toCollections=['GenericNode'])]
    _orphanedCollections = []


class CorpusDistanceAnalyzer(AnalyzerBase):
    def __init__(self, conf: Config):
        super().__init__(conf, run=False)

    def run(self, f_id, t_id):
        query = """FOR c_1 IN Corpus
    FILTER c_1._key==@f_id
    FOR c_2 IN Corpus
        FILTER c_2._key==@t_id
            FOR path IN 1..10 ANY K_PATHS c_1 TO c_2 CorpusHas, HasKeyterm
                LIMIT 1000
                RETURN path"""
        aql = self.database.AQLQuery(query, bindVars={
            'f_id': f_id, 't_id': t_id}, rawResults=True)
        data = list(aql)
        return self.visualize_graph(data,weight_edges=True)


conf = Config(url="http://sc-0050vml:8530/", user='root',
              password='root', graph='KnowledgeGraph',database='SKG-IN')
cd = CorpusDistanceAnalyzer(conf)
g=cd.run('Pangaea','DLR_elib')
g.show("example.html")